<a href="https://colab.research.google.com/github/rajabia/Analyzing-Chicago-Crime-Characteristics/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torchvision
from torchvision import datasets , transforms

In [32]:
from torchvision.datasets.sbd import shutil
transform = transforms.Compose([transforms.ToTensor()])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
print(torch.cuda.is_available())
train_loader = torch.utils.data.DataLoader(dataset= mnist_trainset, batch_size =32, shuffle= True)

False


In [68]:
class AE(torch.nn.Module):
  def __init__(self,use_batch_norm =False, use_dropout = False):
    super().__init__()
    self.use_batch_norm=use_batch_norm
    self.use_dropout=use_dropout
    self.encoder= torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=1, out_channels= 32, kernel_size=3, stride=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=32, out_channels= 64, kernel_size=3, stride=2,padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=64, out_channels= 64, kernel_size=3, stride=2,padding=1),
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels= 64, out_channels= 64, kernel_size=3, stride=1),
        torch.nn.ReLU()

    )

    self.decoder= torch.nn.Sequential(

        torch.nn.ConvTranspose2d(in_channels=64, out_channels= 64, kernel_size=3, stride=1),
        torch.nn.ReLU(),
        torch.nn.ConvTranspose2d(in_channels=64, out_channels= 64, kernel_size=3, stride=2,padding=1),
        torch.nn.ReLU(),
        torch.nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=2,padding=1,output_padding=1 ),
        torch.nn.ReLU(),
        torch.nn.ConvTranspose2d(in_channels= 32 , out_channels= 1, kernel_size=3, stride=1),
        torch.nn.Sigmoid()
    )

  def forward(self, x):
    x= self.encoder(x)
    x= self.decoder(x)
    return x

def train(model, data_loader,num_epochs,learning_rate):
  torch.manual_seed(42)
  criterion = torch.nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,  weight_decay=1e-5)
  for epoch in range(num_epochs):
        for img,_ in train_loader:
            recon = model(img)
            optimizer.zero_grad()
            loss = criterion(recon, img)
            loss.backward()
            optimizer.step()
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
AutoEncod= AE()
# print(AutoEncod)

In [ ]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0

train(AutoEncod, train_loader,5,LEARNING_RATE)